In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
# %matplotlib inline

In [ ]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

In [ ]:
len(words)

In [ ]:
# build vocabulary of characters and mappings to/from indexeger index
chars = sorted(list(set(''.join(words))))
char_to_index = {s:i+1 for i,s in enumerate(chars)}
char_to_index['.'] = 0
index_to_char = {i:s for s,i in char_to_index.items()}
index_to_char

In [ ]:
# build data_set
block_size = 3 # how many chars we view to predict the next one
_input, _label= [],[]

for w in words:
    # print(w)
    context =  [0]*block_size
    for ch in w+'.':
        label_index = char_to_index[ch]
        _input.append(context)
        _label.append(label_index)
        #print(''.join(index_to_char[i] for i in context), ' --->', ch)
        context = context[1:]+[label_index]
        
inputs = torch.tensor(_input)
labels = torch.tensor(_label)

In [30]:
inputs.shape, labels.shape

(torch.Size([32, 3]), torch.Size([32]))

In [35]:
g = torch.Generator().manual_seed(2147483647) 

C = torch.randn((27,2), generator=g) # 27 char, 2 dim per char embedding (feature space)



W1 = torch.randn((6,100) , generator=g) # 3 chars, each in 2 dim
b1 = torch.randn(100, generator=g) # we want to do emb @ W1 + b1


W2 = torch.randn((100,27), generator=g)
b2 = torch.randn(27, generator=g)

parameters = [C, W1, W2, b1, b2]
for p in parameters:
    p.requires_grad = True

In [86]:
for _ in range (10):
    '''Forward Pass'''
    emb = C[inputs] # embed the whole inputs into features

    hidden = torch.tanh(emb.view(-1,6) @ W1 + b1)

    logits = hidden @ W2 + b2 # shape 32, 27
    loss = F.cross_entropy(logits, labels)
    print(loss.item())
    '''Backward Pass'''
    for p in parameters:
        p.grad = None
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad

0.26486366987228394
0.264577716588974
0.2647497057914734
0.2644633650779724
0.26462528109550476
0.2643424868583679
0.2644961476325989
0.26421913504600525
0.26436570286750793
0.2640955448150635
